In [82]:
import pandas as pd
import numpy as np
# some sklearn tools  for  preprocessing and building a pipeline.
# Column transofrmer

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
# For Plotting the shap summary graph 
import matplotlib.pyplot as plt

# To interpret thre model 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import _tree
from xgboost.sklearn import XGBClassifier
import pickle

import lime
from lime import lime_tabular
from lime.lime_tabular import LimeTabularExplainer

# Create progress bar using tqdm module
from tqdm import tqdm

#  over_sampling using SMOTE to balance the data
from imblearn.over_sampling import SMOTE

In [83]:
data_new = pd.read_csv('loan_level_500k.csv')
data_new

,CREDIT_SCORE,FIRST_PAYMENT_DATE,FIRST_TIME_HOMEBUYER_FLAG,MATURITY_DATE,METROPOLITAN_STATISTICAL_AREA,MORTGAGE_INSURANCE_PERCENTAGE,NUMBER_OF_UNITS,OCCUPANCY_STATUS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,...,PROPERTY_TYPE,POSTAL_CODE,LOAN_SEQUENCE_NUMBER,LOAN_PURPOSE,ORIGINAL_LOAN_TERM,NUMBER_OF_BORROWERS,SELLER_NAME,SERVICER_NAME,PREPAID,DELINQUENT
0,669.0,200206,N,202901,NaN,0.0,1.0,O,80.0,33.0,...,SF,26100.0,F199Q1000004,P,320,2.0,Other sellers,Other servicers,True,False
1,732.0,199904,N,202903,17140.0,0.0,1.0,O,25.0,10.0,...,SF,45200.0,F199Q1000005,N,360,1.0,Other sellers,Other servicers,True,False
2,679.0,200208,N,202902,15940.0,30.0,1.0,O,91.0,48.0,...,SF,44700.0,F199Q1000007,P,319,1.0,Other sellers,Other servicers,True,False
3,721.0,200209,N,202902,38060.0,0.0,1.0,O,39.0,13.0,...,SF,85200.0,F199Q1000013,N,318,2.0,Other sellers,Other servicers,True,False
4,618.0,200210,N,202902,10420.0,25.0,1.0,O,85.0,24.0,...,SF,44200.0,F199Q1000015,N,317,2.0,Other sellers,Other servicers,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500132,774.0,200203,NaN,203202,33460.0,0.0,1.0,O,61.0,38.0,...,SF,55400.0,F102Q1125986,C,360,1.0,WELLSFARGOHOMEMORTGA,"WELLSFARGOBANK,NA",True,False
500133,689.0,200203,NaN,203202,10580.0,0.0,1.0,O,70.0,39.0,...,SF,12300.0,F102Q1125989,N,360,1.0,WELLSFARGOHOMEMORTGA,WELLSFARGOHOMEMORTGA,True,False
500134,798.0,200203,NaN,203202,19780.0,0.0,1.0,O,56.0,41.0,...,SF,50300.0,F102Q1125990,C,360,1.0,WELLSFARGOHOMEMORTGA,"WELLSFARGOBANK,NA",True,False
500135,791.0,200203,NaN,203202,42044.0,0.0,1.0,O,26.0,18.0,...,SF,92600.0,F102Q1125991,N,360,1.0,WELLSFARGOHOMEMORTGA,"WELLSFARGOBANK,NA",True,False


In [84]:
# Drop columns
drop_cols = ['FIRST_PAYMENT_DATE', 'MATURITY_DATE', 'MORTGAGE_INSURANCE_PERCENTAGE', 'ORIGINAL_UPB',
             'ORIGINAL_INTEREST_RATE', 'PREPAYMENT_PENALTY_MORTGAGE_FLAG', 'PROPERTY_STATE',
             'LOAN_SEQUENCE_NUMBER', 'SELLER_NAME', 'SERVICER_NAME','NUMBER_OF_BORROWERS']
data_new.drop(columns=drop_cols, inplace=True)

In [85]:
data_new.shape

(500137, 16)

In [86]:
data_new

,CREDIT_SCORE,FIRST_TIME_HOMEBUYER_FLAG,METROPOLITAN_STATISTICAL_AREA,NUMBER_OF_UNITS,OCCUPANCY_STATUS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_LOAN_TO_VALUE,CHANNEL,PRODUCT_TYPE,PROPERTY_TYPE,POSTAL_CODE,LOAN_PURPOSE,ORIGINAL_LOAN_TERM,PREPAID,DELINQUENT
0,669.0,N,NaN,1.0,O,80.0,33.0,80.0,R,FRM,SF,26100.0,P,320,True,False
1,732.0,N,17140.0,1.0,O,25.0,10.0,25.0,R,FRM,SF,45200.0,N,360,True,False
2,679.0,N,15940.0,1.0,O,91.0,48.0,91.0,R,FRM,SF,44700.0,P,319,True,False
3,721.0,N,38060.0,1.0,O,39.0,13.0,39.0,T,FRM,SF,85200.0,N,318,True,False
4,618.0,N,10420.0,1.0,O,85.0,24.0,85.0,R,FRM,SF,44200.0,N,317,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500132,774.0,NaN,33460.0,1.0,O,61.0,38.0,61.0,R,FRM,SF,55400.0,C,360,True,False
500133,689.0,NaN,10580.0,1.0,O,70.0,39.0,70.0,R,FRM,SF,12300.0,N,360,True,False
500134,798.0,NaN,19780.0,1.0,O,56.0,41.0,56.0,R,FRM,SF,50300.0,C,360,True,False
500135,791.0,NaN,42044.0,1.0,O,26.0,18.0,26.0,R,FRM,SF,92600.0,N,360,True,False


In [6]:
# data_new["NUMBER_OF_BORROWERS"] = data_new["NUMBER_OF_BORROWERS"].astype("object")

In [87]:
cat_features= list(data_new.select_dtypes(include=['object','bool']).columns[:-1])

num_features = list(data_new.select_dtypes(include=['int64','float64']).columns[:-1])

In [88]:
#  filling missing values in numerical data 
for i in num_features:
    data_new[i] = data_new[i].fillna(data_new[i].median())

#  filling missing values in categorical data
for i in cat_features:
    data_new[i] = data_new[i].fillna(data_new[i].mode()[0])

In [89]:
replace_dict = {True: 1, 
                False: 0}
data_new.PREPAID.replace(replace_dict, inplace=True)
data_new.PREPAID.value_counts(normalize=True)

1    0.961185
0    0.038815
Name: PREPAID, dtype: float64

In [90]:
bank_data_enco= pd.get_dummies(data_new[cat_features])

In [91]:
bank_data_enco

,PREPAID,FIRST_TIME_HOMEBUYER_FLAG_N,FIRST_TIME_HOMEBUYER_FLAG_Y,OCCUPANCY_STATUS_I,OCCUPANCY_STATUS_O,OCCUPANCY_STATUS_S,CHANNEL_B,CHANNEL_C,CHANNEL_R,CHANNEL_T,PRODUCT_TYPE_FRM,PROPERTY_TYPE_CO,PROPERTY_TYPE_CP,PROPERTY_TYPE_LH,PROPERTY_TYPE_MH,PROPERTY_TYPE_PU,PROPERTY_TYPE_SF,LOAN_PURPOSE_C,LOAN_PURPOSE_N,LOAN_PURPOSE_P
0,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1
1,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0
2,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1
3,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0
4,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500132,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0
500133,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0
500134,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0
500135,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0


In [92]:
std = StandardScaler()

In [93]:
scaled_data = pd.DataFrame((data_new[num_features]),columns=num_features)

In [94]:
scaled_data

,CREDIT_SCORE,METROPOLITAN_STATISTICAL_AREA,NUMBER_OF_UNITS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_LOAN_TO_VALUE,POSTAL_CODE
0,669.0,33340.0,1.0,80.0,33.0,80.0,26100.0
1,732.0,17140.0,1.0,25.0,10.0,25.0,45200.0
2,679.0,15940.0,1.0,91.0,48.0,91.0,44700.0
3,721.0,38060.0,1.0,39.0,13.0,39.0,85200.0
4,618.0,10420.0,1.0,85.0,24.0,85.0,44200.0
...,...,...,...,...,...,...,...
500132,774.0,33460.0,1.0,61.0,38.0,61.0,55400.0
500133,689.0,10580.0,1.0,70.0,39.0,70.0,12300.0
500134,798.0,19780.0,1.0,56.0,41.0,56.0,50300.0
500135,791.0,42044.0,1.0,26.0,18.0,26.0,92600.0


In [95]:
# Independent Variable DataFrame
X = pd.concat([bank_data_enco,scaled_data],axis = 1)
# Dependent Variable(Target Variable)
y = data_new['DELINQUENT']

In [96]:
Xgb_cal = XGBClassifier()

In [133]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state=42)

In [134]:
smote = SMOTE(sampling_strategy='minority')
X_train_resampled,y_train_resampled = smote.fit_resample(X_train,y_train)

In [135]:
# Traning the model
Xgb_cal.fit(X_train_resampled, y_train_resampled)

ValueError: 
All the 20 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/vedant/.local/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3803, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'FIRST_TIME_HOMEBUYER_FLAG'

The above exception was the direct cause of the following

In [100]:
y_pred= Xgb_cal.predict(X_test)

In [101]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [102]:
# checking test data accuracy
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.98      0.99      0.99     96426
        True       0.69      0.50      0.58      3602

    accuracy                           0.97    100028
   macro avg       0.84      0.75      0.79    100028
weighted avg       0.97      0.97      0.97    100028



In [103]:
print(confusion_matrix(y_test, y_pred))

[[95616   810]
 [ 1785  1817]]


In [104]:
test_data_new = pd.DataFrame(X_test)

In [105]:
test_data_new

,PREPAID,FIRST_TIME_HOMEBUYER_FLAG_N,FIRST_TIME_HOMEBUYER_FLAG_Y,OCCUPANCY_STATUS_I,OCCUPANCY_STATUS_O,OCCUPANCY_STATUS_S,CHANNEL_B,CHANNEL_C,CHANNEL_R,CHANNEL_T,...,LOAN_PURPOSE_C,LOAN_PURPOSE_N,LOAN_PURPOSE_P,CREDIT_SCORE,METROPOLITAN_STATISTICAL_AREA,NUMBER_OF_UNITS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_LOAN_TO_VALUE,POSTAL_CODE
254484,1,1,0,0,1,0,0,0,1,0,...,0,0,1,812.0,18580.0,1.0,51.0,17.0,51.0,78300.0
305424,1,1,0,0,1,0,0,0,1,0,...,0,1,0,796.0,14484.0,1.0,70.0,16.0,70.0,2000.0
124448,1,1,0,0,1,0,0,0,0,1,...,0,1,0,690.0,41420.0,1.0,51.0,31.0,51.0,97300.0
220551,1,1,0,0,1,0,0,0,1,0,...,0,0,1,796.0,26420.0,1.0,80.0,31.0,80.0,77400.0
125694,0,1,0,0,1,0,0,0,0,1,...,0,0,1,644.0,12060.0,1.0,90.0,25.0,90.0,30000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413004,1,1,0,0,1,0,0,0,1,0,...,0,0,1,737.0,15980.0,1.0,65.0,64.0,65.0,33900.0
385061,1,1,0,0,1,0,0,0,0,1,...,1,0,0,660.0,41180.0,1.0,80.0,15.0,80.0,63100.0
78859,1,1,0,0,1,0,0,0,1,0,...,0,0,1,773.0,34820.0,1.0,80.0,7.0,80.0,29500.0
482697,1,1,0,0,1,0,0,0,0,1,...,0,1,0,689.0,33340.0,1.0,85.0,35.0,85.0,62400.0


In [106]:
test_data_new['Label'] = y_test

In [107]:
test_data_new['Prediction'] = y_pred

In [108]:
true_positive = test_data_new.copy()
true_positive.reset_index(drop=True, inplace=True)
for i in range(len(true_positive)):
    if true_positive['Label'][i] == 1 and true_positive['Prediction'][i] == 1:
        continue
    else:
        true_positive.drop(i, inplace=True)

In [109]:
true_positive

,PREPAID,FIRST_TIME_HOMEBUYER_FLAG_N,FIRST_TIME_HOMEBUYER_FLAG_Y,OCCUPANCY_STATUS_I,OCCUPANCY_STATUS_O,OCCUPANCY_STATUS_S,CHANNEL_B,CHANNEL_C,CHANNEL_R,CHANNEL_T,...,LOAN_PURPOSE_P,CREDIT_SCORE,METROPOLITAN_STATISTICAL_AREA,NUMBER_OF_UNITS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_LOAN_TO_VALUE,POSTAL_CODE,Label,Prediction
4,0,1,0,0,1,0,0,0,0,1,...,1,644.0,12060.0,1.0,90.0,25.0,90.0,30000.0,True,1
9,0,1,0,0,1,0,0,0,1,0,...,0,647.0,22744.0,1.0,75.0,19.0,75.0,33000.0,True,1
39,0,1,0,0,1,0,0,0,0,1,...,1,695.0,24660.0,1.0,97.0,39.0,97.0,27400.0,True,1
223,0,1,0,0,1,0,0,0,0,1,...,0,598.0,19804.0,1.0,90.0,25.0,90.0,48100.0,True,1
265,0,1,0,0,1,0,0,0,0,1,...,1,663.0,12940.0,1.0,95.0,38.0,95.0,70700.0,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99802,0,1,0,0,1,0,0,0,1,0,...,1,589.0,41700.0,1.0,80.0,24.0,80.0,78200.0,True,1
99883,0,1,0,0,1,0,0,0,0,1,...,0,597.0,19124.0,1.0,88.0,30.0,88.0,75100.0,True,1
99983,0,1,0,0,1,0,0,0,1,0,...,1,603.0,33340.0,1.0,95.0,38.0,95.0,16900.0,True,1
99985,0,1,0,1,0,0,0,0,0,1,...,0,516.0,10420.0,1.0,61.0,40.0,61.0,44300.0,True,1


In [110]:
false_positive = test_data_new.copy()
false_positive.reset_index(drop=True, inplace=True)
for i in range(len(false_positive)):
    if false_positive['Label'][i] == 0 and false_positive['Prediction'][i] == 1:
        continue
    else:
        false_positive.drop(i, inplace=True)

In [111]:
false_positive

,PREPAID,FIRST_TIME_HOMEBUYER_FLAG_N,FIRST_TIME_HOMEBUYER_FLAG_Y,OCCUPANCY_STATUS_I,OCCUPANCY_STATUS_O,OCCUPANCY_STATUS_S,CHANNEL_B,CHANNEL_C,CHANNEL_R,CHANNEL_T,...,LOAN_PURPOSE_P,CREDIT_SCORE,METROPOLITAN_STATISTICAL_AREA,NUMBER_OF_UNITS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_LOAN_TO_VALUE,POSTAL_CODE,Label,Prediction
29,0,1,0,0,1,0,0,0,1,0,...,1,606.0,42644.0,1.0,68.0,30.0,68.0,98100.0,False,1
138,0,1,0,0,1,0,0,0,0,1,...,1,649.0,12060.0,1.0,90.0,21.0,90.0,30300.0,False,1
178,0,1,0,0,1,0,0,0,0,1,...,1,680.0,26420.0,1.0,97.0,28.0,97.0,77300.0,False,1
303,0,1,0,0,1,0,0,0,1,0,...,0,557.0,33340.0,1.0,73.0,34.0,73.0,87800.0,False,1
422,0,1,0,0,1,0,0,0,1,0,...,0,684.0,19660.0,1.0,70.0,32.0,70.0,32700.0,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99546,0,1,0,0,1,0,0,0,0,1,...,1,729.0,12060.0,1.0,88.0,26.0,88.0,30200.0,False,1
99582,0,1,0,0,1,0,0,0,0,1,...,0,651.0,33124.0,1.0,95.0,44.0,95.0,33000.0,False,1
99736,0,1,0,1,0,0,0,0,0,1,...,0,714.0,16974.0,1.0,75.0,57.0,75.0,60500.0,False,1
99775,0,1,0,0,1,0,0,0,1,0,...,0,730.0,19804.0,1.0,74.0,44.0,74.0,48100.0,False,1


In [112]:
true_nagative = test_data_new.copy()
true_nagative.reset_index(drop=True, inplace=True)
for i in range(len(true_nagative)):
    if true_nagative['Label'][i] == 0 and true_nagative['Prediction'][i] == 0:
        continue
    else:
        true_nagative.drop(i, inplace=True)

In [113]:
true_nagative

,PREPAID,FIRST_TIME_HOMEBUYER_FLAG_N,FIRST_TIME_HOMEBUYER_FLAG_Y,OCCUPANCY_STATUS_I,OCCUPANCY_STATUS_O,OCCUPANCY_STATUS_S,CHANNEL_B,CHANNEL_C,CHANNEL_R,CHANNEL_T,...,LOAN_PURPOSE_P,CREDIT_SCORE,METROPOLITAN_STATISTICAL_AREA,NUMBER_OF_UNITS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_LOAN_TO_VALUE,POSTAL_CODE,Label,Prediction
0,1,1,0,0,1,0,0,0,1,0,...,1,812.0,18580.0,1.0,51.0,17.0,51.0,78300.0,False,0
1,1,1,0,0,1,0,0,0,1,0,...,0,796.0,14484.0,1.0,70.0,16.0,70.0,2000.0,False,0
2,1,1,0,0,1,0,0,0,0,1,...,0,690.0,41420.0,1.0,51.0,31.0,51.0,97300.0,False,0
3,1,1,0,0,1,0,0,0,1,0,...,1,796.0,26420.0,1.0,80.0,31.0,80.0,77400.0,False,0
5,1,1,0,0,1,0,0,0,0,1,...,1,684.0,39740.0,1.0,80.0,38.0,80.0,19500.0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100023,1,1,0,0,1,0,0,0,1,0,...,1,737.0,15980.0,1.0,65.0,64.0,65.0,33900.0,False,0
100024,1,1,0,0,1,0,0,0,0,1,...,0,660.0,41180.0,1.0,80.0,15.0,80.0,63100.0,False,0
100025,1,1,0,0,1,0,0,0,1,0,...,1,773.0,34820.0,1.0,80.0,7.0,80.0,29500.0,False,0
100026,1,1,0,0,1,0,0,0,0,1,...,0,689.0,33340.0,1.0,85.0,35.0,85.0,62400.0,False,0


In [114]:
false_nagative = test_data_new.copy()
false_nagative.reset_index(drop=True, inplace=True)
for i in range(len(false_nagative)):
    if false_nagative['Label'][i] == 1 and false_nagative['Prediction'][i] == 0:
        continue
    else:
        false_nagative.drop(i, inplace=True)

In [115]:
false_nagative

,PREPAID,FIRST_TIME_HOMEBUYER_FLAG_N,FIRST_TIME_HOMEBUYER_FLAG_Y,OCCUPANCY_STATUS_I,OCCUPANCY_STATUS_O,OCCUPANCY_STATUS_S,CHANNEL_B,CHANNEL_C,CHANNEL_R,CHANNEL_T,...,LOAN_PURPOSE_P,CREDIT_SCORE,METROPOLITAN_STATISTICAL_AREA,NUMBER_OF_UNITS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_LOAN_TO_VALUE,POSTAL_CODE,Label,Prediction
23,0,0,1,0,1,0,0,0,1,0,...,1,724.0,35644.0,1.0,80.0,47.0,80.0,7600.0,True,0
26,0,0,1,0,1,0,0,0,0,1,...,1,696.0,35084.0,1.0,77.0,40.0,77.0,7000.0,True,0
38,0,0,1,0,1,0,0,0,0,1,...,1,719.0,13644.0,1.0,95.0,33.0,95.0,20800.0,True,0
64,1,1,0,0,1,0,0,0,1,0,...,0,300.0,33780.0,1.0,93.0,33.0,93.0,48100.0,True,0
67,1,1,0,0,1,0,0,0,0,1,...,1,626.0,38900.0,1.0,95.0,23.0,95.0,97000.0,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99878,1,1,0,0,1,0,0,0,0,1,...,0,636.0,24500.0,1.0,78.0,30.0,78.0,59400.0,True,0
99917,0,1,0,0,1,0,0,0,1,0,...,0,701.0,24860.0,1.0,95.0,47.0,95.0,29600.0,True,0
99929,1,1,0,0,1,0,0,0,0,1,...,1,681.0,42644.0,1.0,90.0,37.0,90.0,98200.0,True,0
99965,0,1,0,0,1,0,0,0,1,0,...,1,762.0,33340.0,1.0,90.0,41.0,90.0,89400.0,True,0


In [116]:
true_positive.reset_index(drop=True,inplace=True)
false_positive.reset_index(drop=True,inplace=True)
true_nagative.reset_index(drop=True,inplace=True)
false_nagative.reset_index(drop=True,inplace=True)

In [117]:
true_positive.drop(['Label','Prediction'],axis=1,inplace=True)
false_positive.drop(['Label','Prediction'],axis=1,inplace=True)
true_nagative.drop(['Label','Prediction'],axis=1,inplace=True)
false_nagative.drop(['Label','Prediction'],axis=1,inplace=True)

In [118]:
explainer = lime_tabular.LimeTabularExplainer(
            training_data=np.array(X_train_resampled),
            feature_names=X_train_resampled.columns,
            class_names=[0,1],
            mode='classification'
)

In [119]:
explainer.explain_instance(data_row=false_positive.loc[0], predict_fn=Xgb_cal.predict_proba).local_exp

{1: [(19, -0.06453979776932213),
  (0, 0.053742445303995766),
  (18, 0.05019274055566305),
  (8, -0.04989882342024555),
  (11, 0.049117681793204394),
  (9, 0.046131349547752214),
  (15, 0.04315187770092551),
  (17, 0.035130870672529346),
  (12, -0.03331914738974039),
  (6, -0.025585587156993814)]}

In [120]:
feature_name_list = []
importance_value_list = []
for i in tqdm(range(len(false_positive))):
    exp = explainer.explain_instance(data_row=false_positive.iloc[i], predict_fn=Xgb_cal.predict_proba)
    feature_name_list.append(false_positive.columns[exp.local_exp[1][0][0]])
    importance_value_list.append(exp.local_exp[1][0][1])

100%|█████████████████████████████████████████| 810/810 [10:37<00:00,  1.27it/s]


In [121]:
feature_name_list

['LOAN_PURPOSE_P',
 'PROPERTY_TYPE_CO',
 'PROPERTY_TYPE_LH',
 'CHANNEL_C',
 'PREPAID',
 'PROPERTY_TYPE_MH',
 'PROPERTY_TYPE_MH',
 'PROPERTY_TYPE_CO',
 'PROPERTY_TYPE_CP',
 'LOAN_PURPOSE_P',
 'LOAN_PURPOSE_P',
 'LOAN_PURPOSE_P',
 'PROPERTY_TYPE_LH',
 'CHANNEL_B',
 'PREPAID',
 'PROPERTY_TYPE_LH',
 'PROPERTY_TYPE_MH',
 'PROPERTY_TYPE_MH',
 'CHANNEL_C',
 'PREPAID',
 'PREPAID',
 'LOAN_PURPOSE_P',
 'LOAN_PURPOSE_P',
 'CHANNEL_B',
 'LOAN_PURPOSE_P',
 'LOAN_PURPOSE_P',
 'LOAN_PURPOSE_P',
 'PROPERTY_TYPE_CP',
 'LOAN_PURPOSE_P',
 'LOAN_PURPOSE_N',
 'CHANNEL_B',
 'PROPERTY_TYPE_MH',
 'LOAN_PURPOSE_P',
 'PROPERTY_TYPE_LH',
 'PROPERTY_TYPE_LH',
 'PROPERTY_TYPE_LH',
 'CHANNEL_B',
 'PROPERTY_TYPE_MH',
 'CHANNEL_R',
 'PROPERTY_TYPE_CP',
 'LOAN_PURPOSE_P',
 'LOAN_PURPOSE_P',
 'CHANNEL_C',
 'CHANNEL_B',
 'LOAN_PURPOSE_N',
 'LOAN_PURPOSE_N',
 'LOAN_PURPOSE_P',
 'LOAN_PURPOSE_P',
 'PROPERTY_TYPE_CO',
 'PROPERTY_TYPE_LH',
 'PROPERTY_TYPE_CO',
 'PROPERTY_TYPE_CP',
 'CHANNEL_C',
 'PROPERTY_TYPE_CP',
 'PROPER

In [122]:
importance_value_list

[-0.06268817707936664,
 0.06065398208352763,
 0.25838076011489697,
 0.16958307335195635,
 0.057225438054949646,
 0.07813022716478389,
 0.09105297504903755,
 0.06072777835052041,
 0.2087802455675266,
 0.05738124027440095,
 0.05806404424119891,
 0.06220072894951995,
 0.18335653121439865,
 0.15733072339184187,
 0.06009103680837938,
 0.3449838739105467,
 0.10444611514834963,
 0.07975984193433151,
 0.15868429138113385,
 0.060328125072768406,
 0.06027361145728709,
 -0.06172809731711182,
 0.06270943838599173,
 0.33612962447733974,
 -0.05406451346272535,
 0.06454535222972156,
 0.06205612124359984,
 0.17770440410110955,
 -0.06207734768303239,
 -0.06317042369678579,
 0.24500270191786439,
 0.1435178783973664,
 0.0665026099476922,
 0.11609199620313763,
 0.11805569922986899,
 0.250422113655261,
 0.15886739882862222,
 0.15822279477755521,
 0.060775673140803045,
 0.15969737305431703,
 -0.059924327164930874,
 -0.06405677513903349,
 0.1529028504056058,
 -0.07541952914109984,
 0.06604705555493505,
 -0.0

In [123]:
len(feature_name_list)

810

In [124]:
len(importance_value_list)

810

In [45]:
feature_name_list_1 = []
importance_value_list_1 = []
for i in tqdm(range(len(false_nagative))):
    exp_1 = explainer.explain_instance(data_row=false_nagative.iloc[i], predict_fn=Xgb_cal.predict_proba)
    feature_name_list_1.append(false_nagative.columns[exp_1.local_exp[1][0][0]])
    importance_value_list_1.append(exp_1.local_exp[1][0][1])

100%|███████████████████████████████████████| 1768/1768 [04:51<00:00,  6.06it/s]


In [46]:
feature_name_list_1

['NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_CP',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_B',
 'CHANNEL_C',
 'PROPERTY_TYPE_MH',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_B',
 'CHANNEL_B',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_MH',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_B',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_LH',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_C',
 'PROPERTY_TYPE_LH',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_C',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_CP',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_MH',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_CP',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_LH',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_CP',
 'PROPERTY_TYPE_MH',
 'PRO

In [47]:
importance_value_list_1

[-0.060165807655523366,
 -0.055363454027172326,
 -0.054146382902662084,
 0.036425232924754775,
 0.07765949968777226,
 0.04554903098373816,
 0.21298280234531877,
 0.10542524413450612,
 0.08678642203716584,
 -0.061487918522158,
 0.2390969646572768,
 0.06376894001611036,
 -0.0675059725358041,
 0.06319844476470492,
 -0.05696838482207536,
 0.18723639491518704,
 0.08292418981188815,
 0.04472287422944368,
 -0.04301448023425547,
 0.12128085777068576,
 -0.06262194802926191,
 -0.05585126669586154,
 -0.058012980718704754,
 0.04330548512139815,
 0.12025988926124363,
 0.09709861546676794,
 -0.0636069358770759,
 0.040112313496725546,
 -0.05488207881214667,
 0.054096191056330144,
 0.048950651787678906,
 -0.060824279196653376,
 0.04214081859453493,
 -0.06009428689379397,
 0.041107075329422905,
 0.040945604649187825,
 0.07657602711340636,
 0.04564271796705319,
 0.0516382215425483,
 -0.05675258527259809,
 0.2419540308673512,
 -0.05765242918291167,
 0.10875805626845293,
 -0.05974015334355203,
 -0.0664809

In [48]:
len(feature_name_list_1)

1768

In [49]:
len(importance_value_list_1)

1768

In [50]:
Xgb_cal.feature_importances_

array([0.12388473, 0.3422003 , 0.00917312, 0.05122187, 0.01295372,
       0.0047331 , 0.01649331, 0.00326624, 0.00448919, 0.02863507,
       0.07135927, 0.        , 0.04737812, 0.0074422 , 0.00444747,
       0.01573685, 0.01285112, 0.00921881, 0.12416138, 0.03190716,
       0.0311869 , 0.01672001, 0.00094576, 0.00636322, 0.00579093,
       0.00738297, 0.00872899, 0.00132833], dtype=float32)

In [51]:
set(feature_name_list)

{'CHANNEL_B',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'OCCUPANCY_STATUS_S',
 'PROPERTY_TYPE_CO',
 'PROPERTY_TYPE_CP',
 'PROPERTY_TYPE_LH',
 'PROPERTY_TYPE_MH'}

In [52]:
len(set(feature_name_list))

8

In [53]:
list(set(feature_name_list))

['PROPERTY_TYPE_CO',
 'CHANNEL_B',
 'OCCUPANCY_STATUS_S',
 'PROPERTY_TYPE_CP',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_C',
 'PROPERTY_TYPE_MH',
 'PROPERTY_TYPE_LH']

In [54]:
len(list(set(feature_name_list)))

8

In [55]:
set(feature_name_list_1)

{'CHANNEL_B',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'OCCUPANCY_STATUS_S',
 'PROPERTY_TYPE_CO',
 'PROPERTY_TYPE_CP',
 'PROPERTY_TYPE_LH',
 'PROPERTY_TYPE_MH',
 'PROPERTY_TYPE_PU'}

In [56]:
len(set(feature_name_list_1))

9

In [57]:
list(set(feature_name_list_1))

['PROPERTY_TYPE_CO',
 'PROPERTY_TYPE_PU',
 'CHANNEL_B',
 'OCCUPANCY_STATUS_S',
 'PROPERTY_TYPE_CP',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_C',
 'PROPERTY_TYPE_MH',
 'PROPERTY_TYPE_LH']

In [58]:
len(list(set(feature_name_list_1)))

9

In [59]:
# feature_name_list_1 and feature_name_list in these list, take mode of it and then remove that feature from dataset and retrain the model and check accuracy.

In [66]:
feature_name_list

['NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_MH',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_C',
 'PROPERTY_TYPE_MH',
 'PROPERTY_TYPE_MH',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_B',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_B',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_LH',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_CP',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_CP',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_CP',
 'PROPERTY_TYPE_CP',
 'PROPERTY_TYPE_MH',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORRO

In [71]:
feature_name_list_1

['NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_CP',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_B',
 'CHANNEL_C',
 'PROPERTY_TYPE_MH',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_B',
 'CHANNEL_B',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_MH',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_B',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_LH',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_C',
 'PROPERTY_TYPE_LH',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'CHANNEL_C',
 'CHANNEL_C',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_CP',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_MH',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_CP',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_LH',
 'NUMBER_OF_BORROWERS',
 'NUMBER_OF_BORROWERS',
 'PROPERTY_TYPE_CP',
 'PROPERTY_TYPE_MH',
 'PRO

In [67]:
# main_list = {
#                 "feature_name":feature_name_list,
#                 "feature_name_list":feature_name_list_1
# }

In [78]:
main1 = pd.DataFrame(feature_name_list,columns=["Feature_Name"]).mode()

In [79]:
main2 = pd.DataFrame(feature_name_list_1,columns=["Feature_Name_List"]).mode()

In [80]:
main1

,Feature_Name
0,NUMBER_OF_BORROWERS


In [81]:
main2

,Feature_Name_List
0,NUMBER_OF_BORROWERS
